# 线性回归的简单实现

## 生成数据集

In [1]:
from mxnet import autograd, nd

In [5]:
num_examples = 1000
num_inputs = 2

true_w = nd.array([2, -3.4])
true_b = 4.2

features = nd.random.normal(scale=1, shape=(num_examples, num_inputs))
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b

labels += nd.random.normal(scale=0.01, shape=labels.shape)

## 读取数据集

In [6]:
from mxnet.gluon import data as gdata

In [7]:
batch_size = 10
# 组合特征和标签
dataset = gdata.ArrayDataset(features, labels)
dataset = gdata.DataLoader(dataset, batch_size,shuffle=True)

In [8]:
for x, y in dataset:
    print(x, y)
    break


[[ 0.4308203  -1.2022533 ]
 [-0.04744347  0.34213507]
 [-0.01285097  0.88687867]
 [ 0.2984833  -1.1776266 ]
 [-0.19438495  0.32892177]
 [ 0.47146404 -1.3202714 ]
 [ 1.1482935   1.2560791 ]
 [-0.46596846 -1.9588767 ]
 [-0.00766342 -1.393498  ]
 [ 0.3624834   0.00751651]]
<NDArray 10x2 @cpu(0)> 
[9.133583  2.9460514 1.1784701 8.794146  2.6969435 9.640236  2.2375505
 9.907577  8.926412  4.9114113]
<NDArray 10 @cpu(0)>


## 定义模型

In [9]:
from mxnet.gluon import nn

net = nn.Sequential()

In [10]:
net.add(nn.Dense(1))

## 初始化模型参数

In [11]:
from mxnet import init

net.initialize(init.Normal(sigma=0.01))

## 定义损失函数

In [12]:
from mxnet.gluon import loss as gloss

loss = gloss.L2Loss() #平方损失函数，又称L2范数损失函数

## 定义优化算法

In [14]:
from mxnet import gluon

trainer = gluon.Trainer(net.collect_params(), 'sgd',{'learning_rate':0.03})

## 训练模型

In [18]:
num_epochs = 3
for epoch in range(num_epochs):
    for x, y in dataset:
        with autograd.record():
            l = loss(net(x), y)
        l.backward()
        trainer.step(batch_size)
    train_l = loss(net(features), labels)
    print('epoch %d, loss %f' % (epoch + 1, train_l.mean().asnumpy()))

epoch 1, loss 0.000142
epoch 2, loss 0.000050
epoch 3, loss 0.000050


In [19]:
dense = net[0]
print(true_w, dense.weight.data())

(
 [ 2.  -3.4]
 <NDArray 2 @cpu(0)>, 
 [[ 2.0000582 -3.399764 ]]
 <NDArray 1x2 @cpu(0)>)

In [21]:
print(true_b, dense.bias.data())

4.2 
[4.2006707]
<NDArray 1 @cpu(0)>
